In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm 

### #1: Dataset Class and Dataloader

In [2]:
class PalmsDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir,transform=transform)
        self.transform=transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]
    
    @property
    def classes(self):
        return self.data.classes

In [3]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
])

In [ ]:
train_set = PalmsDataset("train",transform=train_transform)

#### Dataset vs Dataloader

In [5]:
for image, label in train_set:
    break

image.shape, label

(torch.Size([3, 224, 224]), 0)

In [6]:
train_loader = DataLoader(batch_size=32, dataset=train_set, shuffle=True)

### #2: Classifier

In [ ]:
class PalmDiseaseClassifier(nn.Module):
    def __init__(self, number_of_classes=9):
        super(PalmDiseaseClassifier, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1, stride=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1, stride=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1, stride=1)
        
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.drop1 = nn.Dropout(0.2)
        self.fc1 = nn.Linear(128*28*28, number_of_classes)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        # print(x.shape) # to get the needed input size for fc1
        
        x = x.view(x.size(0), -1)
        # print(x.shape)
        x = self.drop1(x)
        x = self.fc1(x)
        
        return x
        
        
        

In [ ]:
model = PalmDiseaseClassifier()

In [9]:
# the needed x shape for fc1
# add print(x.shape) after the pool, then comment the fc1 and run the model on a batch
# the output will be [batch size, channel, highet, width]. the input for fc will be C*H*W
for x, y in train_loader:
    break
model(x)

tensor([[-7.0882e-02, -5.1109e-01,  7.1248e-01, -6.6899e-01,  2.6139e-01,
          1.6226e-01, -6.4960e-01,  1.0716e+00, -4.4262e-01],
        [-5.8032e-01, -1.0455e+00,  1.8994e-01, -2.8990e-01,  3.7907e-01,
         -6.9123e-01, -1.3212e-01, -8.6309e-02,  7.4228e-01],
        [-8.0366e-01, -5.7059e-02,  1.1858e-02, -3.1909e-01, -1.2748e-01,
          1.0562e+00, -1.0301e-01, -2.3994e-01,  2.0301e-01],
        [-1.3421e-01,  2.9368e-01,  7.9003e-01, -2.7535e-01, -3.2956e-01,
         -2.6141e-01,  1.6692e-01, -3.0124e-01, -7.2717e-01],
        [-3.5331e-01,  1.6447e-01,  3.0154e-01, -1.1321e+00,  2.6669e-01,
          4.8662e-01, -7.4506e-01,  4.9547e-01, -4.0434e-01],
        [-5.3650e-01, -1.4516e-01,  3.3962e-01,  2.7971e-01,  5.5072e-01,
          1.7346e-01, -8.1457e-03,  2.3874e-01,  3.8237e-01],
        [-6.7001e-01, -9.7767e-01,  3.0849e-01, -1.2333e+00, -4.7234e-01,
          8.2979e-02, -3.1822e-01,  1.0678e+00,  3.2823e-01],
        [-2.4868e-01, -2.9906e-01,  4.7976e-01, 

In [13]:
print(str(model))

PalmClassifier(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=100352, out_features=9, bias=True)
)


In [14]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

In [ ]:
val_dataset = PalmsDataset("valid", transform=transform)
test_dataset = PalmsDataset("test", transform=transform)

val_loader = DataLoader(val_dataset,batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
device = torch.device("cuda:0")

In [ ]:
model = PalmDiseaseClassifier(number_of_classes=9).to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=25):
    best_acc = 0.0
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} (Train)"):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total
        
        val_loss, val_acc = validate(model, val_loader, criterion)
        
        print(f"Epoch {epoch+1}: "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), 'best_palm_cnn_temp.pth')
            print(f"Saved new best model (Acc: {best_acc:.2f}%)")

def validate(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_loss = running_loss / len(val_loader)
    val_acc = 100 * correct / total
    return val_loss, val_acc

train_model(model, train_loader, val_loader, criterion, optimizer, epochs=50)

Validating: 100%|██████████| 19/19 [00:05<00:00,  3.59it/s]


Epoch 1: Train Loss: 2.4977, Train Acc: 33.96% | Val Loss: 2.1183, Val Acc: 41.18%
Saved new best model (Acc: 41.18%)


Validating: 100%|██████████| 19/19 [00:01<00:00, 10.27it/s]


Epoch 2: Train Loss: 1.5807, Train Acc: 46.73% | Val Loss: 1.3979, Val Acc: 57.96%
Saved new best model (Acc: 57.96%)


Validating: 100%|██████████| 19/19 [00:01<00:00,  9.90it/s]


...

Epoch 48: Train Loss: 0.1691, Train Acc: 93.96% | Val Loss: 1.1335, Val Acc: 75.78%


Validating: 100%|██████████| 19/19 [00:01<00:00, 10.14it/s]


Epoch 49: Train Loss: 0.1593, Train Acc: 94.36% | Val Loss: 1.2050, Val Acc: 73.18%


Validating: 100%|██████████| 19/19 [00:01<00:00,  9.68it/s]

Epoch 50: Train Loss: 0.1982, Train Acc: 93.66% | Val Loss: 1.1035, Val Acc: 76.99%
